In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 연봉데이터 로딩, 전처리, eda, feature engineering, modeling, 성능 검증, 하이퍼파라미터 튜닝

## 기본 모델은 decisionTree -> 성능 기준

## 더 잘 나오기 위해 배깅, 부스팅, 랜덤배깅 모델을 튜닝

### 분석의 목적은 학력, 교육, 연수, 혼인 상태, 직업정보가 있는 연봉데이터셋을 이용해 연봉 예측하기
### 연봉이 5만 달러 이상인지 아닌지 예측하는 것

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/haram4th/ADsP/main/salary2.csv")
data

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       46043 non-null  object
 2   education       48842 non-null  object
 3   education-num   48842 non-null  int64 
 4   marital-status  48842 non-null  object
 5   occupation      46033 non-null  object
 6   relationship    48842 non-null  object
 7   race            48842 non-null  object
 8   sex             48842 non-null  object
 9   capital-gain    48842 non-null  int64 
 10  capital-loss    48842 non-null  int64 
 11  hours-per-week  48842 non-null  int64 
 12  native-country  47985 non-null  object
 13  class           48842 non-null  object
dtypes: int64(5), object(9)
memory usage: 5.2+ MB


In [4]:
print('고용형태', '\n', data['workclass'].unique())
print('학력', '\n', data['education'].unique())
print('혼인상태', '\n', data['marital-status'].unique())
print('가족관계', '\n', data['relationship'].unique())
print('인종', '\n', data['race'].unique())
print('본국', '\n', data['native-country'].unique())

고용형태 
 [' Private' ' Local-gov' nan ' Self-emp-not-inc' ' Federal-gov'
 ' State-gov' ' Self-emp-inc' ' Without-pay' ' Never-worked']
학력 
 [' 11th' ' HS-grad' ' Assoc-acdm' ' Some-college' ' 10th' ' Prof-school'
 ' 7th-8th' ' Bachelors' ' Masters' ' Doctorate' ' 5th-6th' ' Assoc-voc'
 ' 9th' ' 12th' ' 1st-4th' ' Preschool']
혼인상태 
 [' Never-married' ' Married-civ-spouse' ' Widowed' ' Divorced'
 ' Separated' ' Married-spouse-absent' ' Married-AF-spouse']
가족관계 
 [' Own-child' ' Husband' ' Not-in-family' ' Unmarried' ' Wife'
 ' Other-relative']
인종 
 [' Black' ' White' ' Asian-Pac-Islander' ' Other' ' Amer-Indian-Eskimo']
본국 
 [' United-States' nan ' Peru' ' Guatemala' ' Mexico' ' Dominican-Republic'
 ' Ireland' ' Germany' ' Philippines' ' Thailand' ' Haiti' ' El-Salvador'
 ' Puerto-Rico' ' Vietnam' ' South' ' Columbia' ' Japan' ' India'
 ' Cambodia' ' Poland' ' Laos' ' England' ' Cuba' ' Taiwan' ' Italy'
 ' Canada' ' Portugal' ' China' ' Nicaragua' ' Honduras' ' Iran'
 ' Scotland' ' Jamaica

In [5]:
data['education-num'].value_counts()

education-num
9     15784
10    10878
13     8025
14     2657
11     2061
7      1812
12     1601
6      1389
4       955
15      834
5       756
8       657
16      594
3       509
2       247
1        83
Name: count, dtype: int64

In [6]:
data.describe()

,age,education-num,capital-gain,capital-loss,hours-per-week
count,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000
mean,38.643585,10.078089,1079.067626,87.502314,40.422382
std,13.710510,2.570973,7452.019058,403.004552,12.391444
min,17.000000,1.000000,0.000000,0.000000,1.000000
25%,28.000000,9.000000,0.000000,0.000000,40.000000
50%,37.000000,10.000000,0.000000,0.000000,40.000000
75%,48.000000,12.000000,0.000000,0.000000,45.000000
max,90.000000,16.000000,99999.000000,4356.000000,99.000000


#### 결측값이 있는 값은 workclass, occupation, native-country

#### workclass 결측값 처리

In [7]:
data['workclass'].value_counts()

workclass
Private             33906
Self-emp-not-inc     3862
Local-gov            3136
State-gov            1981
Self-emp-inc         1695
Federal-gov          1432
Without-pay            21
Never-worked           10
Name: count, dtype: int64

In [8]:
data[data['workclass'].isna()]

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
4,18,NaN,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K
6,29,NaN,HS-grad,9,Never-married,NaN,Unmarried,Black,Male,0,0,40,United-States,<=50K
13,58,NaN,HS-grad,9,Married-civ-spouse,NaN,Husband,White,Male,0,0,35,United-States,<=50K
22,72,NaN,7th-8th,4,Divorced,NaN,Not-in-family,White,Female,0,0,6,United-States,<=50K
35,65,NaN,HS-grad,9,Married-civ-spouse,NaN,Husband,White,Male,0,0,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48811,35,NaN,Bachelors,13,Married-civ-spouse,NaN,Wife,White,Female,0,0,55,United-States,>50K
48812,30,NaN,Bachelors,13,Never-married,NaN,Not-in-family,Asian-Pac-Islander,Female,0,0,99,United-States,<=50K
48820,71,NaN,Doctorate,16,Married-civ-spouse,NaN,Husband,White,Male,0,0,10,United-States,>50K
48822,41,NaN,HS-grad,9,Separated,NaN,Not-in-family,Black,Female,0,0,32,United-States,<=50K


In [9]:
2799 / 48842 * 100

5.7307235575938735

workclass의 결측값은 2799개

In [10]:
# 고용형태가 없다면 직업도 없을 가능성이 크므로 workclass그룹별 occupation을 조회함
data[data['workclass'].isna()][['workclass', 'occupation']].describe()

,workclass,occupation
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


직업이 없는 사람은 고용형태가 없다고 판단해 workclass와 occupation 모두 결측값인 경우 none으로 처리

In [11]:
data[(data['workclass'].isna()) & (data['occupation'].isna())]

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
4,18,NaN,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K
6,29,NaN,HS-grad,9,Never-married,NaN,Unmarried,Black,Male,0,0,40,United-States,<=50K
13,58,NaN,HS-grad,9,Married-civ-spouse,NaN,Husband,White,Male,0,0,35,United-States,<=50K
22,72,NaN,7th-8th,4,Divorced,NaN,Not-in-family,White,Female,0,0,6,United-States,<=50K
35,65,NaN,HS-grad,9,Married-civ-spouse,NaN,Husband,White,Male,0,0,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48811,35,NaN,Bachelors,13,Married-civ-spouse,NaN,Wife,White,Female,0,0,55,United-States,>50K
48812,30,NaN,Bachelors,13,Never-married,NaN,Not-in-family,Asian-Pac-Islander,Female,0,0,99,United-States,<=50K
48820,71,NaN,Doctorate,16,Married-civ-spouse,NaN,Husband,White,Male,0,0,10,United-States,>50K
48822,41,NaN,HS-grad,9,Separated,NaN,Not-in-family,Black,Female,0,0,32,United-States,<=50K


In [12]:
# 테이블을 따로 지정
nas = data[(data['workclass'].isna()) & (data['occupation'].isna())]
nas

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
4,18,NaN,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K
6,29,NaN,HS-grad,9,Never-married,NaN,Unmarried,Black,Male,0,0,40,United-States,<=50K
13,58,NaN,HS-grad,9,Married-civ-spouse,NaN,Husband,White,Male,0,0,35,United-States,<=50K
22,72,NaN,7th-8th,4,Divorced,NaN,Not-in-family,White,Female,0,0,6,United-States,<=50K
35,65,NaN,HS-grad,9,Married-civ-spouse,NaN,Husband,White,Male,0,0,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48811,35,NaN,Bachelors,13,Married-civ-spouse,NaN,Wife,White,Female,0,0,55,United-States,>50K
48812,30,NaN,Bachelors,13,Never-married,NaN,Not-in-family,Asian-Pac-Islander,Female,0,0,99,United-States,<=50K
48820,71,NaN,Doctorate,16,Married-civ-spouse,NaN,Husband,White,Male,0,0,10,United-States,>50K
48822,41,NaN,HS-grad,9,Separated,NaN,Not-in-family,Black,Female,0,0,32,United-States,<=50K


In [13]:
nas['workclass'] = nas['workclass'].fillna("None")
nas['occupation'] = nas['occupation'].fillna("None")
nas

C:\Users\5-08\AppData\Local\Temp\ipykernel_12408\3962614923.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nas['workclass'] = nas['workclass'].fillna("None")
C:\Users\5-08\AppData\Local\Temp\ipykernel_12408\3962614923.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nas['occupation'] = nas['occupation'].fillna("None")


,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
4,18,None,Some-college,10,Never-married,None,Own-child,White,Female,0,0,30,United-States,<=50K
6,29,None,HS-grad,9,Never-married,None,Unmarried,Black,Male,0,0,40,United-States,<=50K
13,58,None,HS-grad,9,Married-civ-spouse,None,Husband,White,Male,0,0,35,United-States,<=50K
22,72,None,7th-8th,4,Divorced,None,Not-in-family,White,Female,0,0,6,United-States,<=50K
35,65,None,HS-grad,9,Married-civ-spouse,None,Husband,White,Male,0,0,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48811,35,None,Bachelors,13,Married-civ-spouse,None,Wife,White,Female,0,0,55,United-States,>50K
48812,30,None,Bachelors,13,Never-married,None,Not-in-family,Asian-Pac-Islander,Female,0,0,99,United-States,<=50K
48820,71,None,Doctorate,16,Married-civ-spouse,None,Husband,White,Male,0,0,10,United-States,>50K
48822,41,None,HS-grad,9,Separated,None,Not-in-family,Black,Female,0,0,32,United-States,<=50K


In [14]:
data[(data['workclass'].isna()) & (data['occupation'].isna())] = nas
data[data['workclass'].isna()]

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       48842 non-null  object
 2   education       48842 non-null  object
 3   education-num   48842 non-null  int64 
 4   marital-status  48842 non-null  object
 5   occupation      48832 non-null  object
 6   relationship    48842 non-null  object
 7   race            48842 non-null  object
 8   sex             48842 non-null  object
 9   capital-gain    48842 non-null  int64 
 10  capital-loss    48842 non-null  int64 
 11  hours-per-week  48842 non-null  int64 
 12  native-country  47985 non-null  object
 13  class           48842 non-null  object
dtypes: int64(5), object(9)
memory usage: 5.2+ MB


#### workclass 의 결측값은 모두 직업이 없는 경우이기 때문에 workclass와 occupation 모두 none으로 처리

#### occupation 결측값 처리

In [16]:
data[data['occupation'].isna()]

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
8785,17,Never-worked,11th,7,Never-married,NaN,Own-child,Black,Female,0,0,20,United-States,<=50K
11607,20,Never-worked,HS-grad,9,Married-spouse-absent,NaN,Other-relative,White,Male,0,0,35,United-States,<=50K
13898,18,Never-worked,11th,7,Never-married,NaN,Own-child,White,Male,0,0,35,United-States,<=50K
21642,18,Never-worked,10th,6,Never-married,NaN,Own-child,White,Male,0,0,40,United-States,<=50K
27126,23,Never-worked,7th-8th,4,Divorced,NaN,Not-in-family,White,Male,0,0,35,United-States,<=50K
31053,17,Never-worked,10th,6,Never-married,NaN,Own-child,White,Male,0,0,30,United-States,<=50K
36618,18,Never-worked,11th,7,Never-married,NaN,Own-child,White,Female,0,0,10,United-States,<=50K
39513,20,Never-worked,Some-college,10,Never-married,NaN,Own-child,Black,Male,0,0,40,United-States,<=50K
48585,30,Never-worked,HS-grad,9,Married-civ-spouse,NaN,Wife,Black,Female,0,0,40,United-States,<=50K
48595,18,Never-worked,Some-college,10,Never-married,NaN,Own-child,White,Male,0,0,4,United-States,<=50K


#### 이 경우 occupation이 결측값인 이유는 workclass가 never-worked인 경우임
#### never-worked는 한 번도 일을 해 본 적이 없다는 경우이기에 직업이 없는 것이 맞음
#### 즉, 이 경우에도 none으로 처리할 예정

In [17]:
data['occupation'] = data['occupation'].fillna('None')
data[data['occupation'].isna()]

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       48842 non-null  object
 2   education       48842 non-null  object
 3   education-num   48842 non-null  int64 
 4   marital-status  48842 non-null  object
 5   occupation      48842 non-null  object
 6   relationship    48842 non-null  object
 7   race            48842 non-null  object
 8   sex             48842 non-null  object
 9   capital-gain    48842 non-null  int64 
 10  capital-loss    48842 non-null  int64 
 11  hours-per-week  48842 non-null  int64 
 12  native-country  47985 non-null  object
 13  class           48842 non-null  object
dtypes: int64(5), object(9)
memory usage: 5.2+ MB


#### occupation 결측값 처리 완료
#### native-country 결측값 처리하기

In [19]:
data[data['native-country'].isna()]

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
19,40,Private,Doctorate,16,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,45,NaN,>50K
65,41,Private,Bachelors,13,Never-married,Other-service,Not-in-family,White,Female,0,0,40,NaN,<=50K
83,44,Self-emp-inc,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,99999,0,50,NaN,>50K
188,34,State-gov,Bachelors,13,Married-spouse-absent,Farming-fishing,Not-in-family,Black,Male,0,0,40,NaN,<=50K
253,42,Federal-gov,Bachelors,13,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,40,NaN,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48730,44,Self-emp-inc,Masters,14,Married-civ-spouse,Sales,Husband,White,Male,0,0,50,NaN,>50K
48750,58,Self-emp-inc,Doctorate,16,Never-married,Prof-specialty,Not-in-family,White,Female,0,0,99,NaN,<=50K
48773,42,Self-emp-not-inc,HS-grad,9,Divorced,Sales,Own-child,White,Male,0,0,50,NaN,<=50K
48791,39,Private,HS-grad,9,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,45,NaN,>50K


In [20]:
# 본국이 결측값인 경우 1.75%
857 / 48842 * 100

1.7546374022357807

In [21]:
a = data[data['native-country'].isna()]
a

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
19,40,Private,Doctorate,16,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,45,NaN,>50K
65,41,Private,Bachelors,13,Never-married,Other-service,Not-in-family,White,Female,0,0,40,NaN,<=50K
83,44,Self-emp-inc,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,99999,0,50,NaN,>50K
188,34,State-gov,Bachelors,13,Married-spouse-absent,Farming-fishing,Not-in-family,Black,Male,0,0,40,NaN,<=50K
253,42,Federal-gov,Bachelors,13,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,40,NaN,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48730,44,Self-emp-inc,Masters,14,Married-civ-spouse,Sales,Husband,White,Male,0,0,50,NaN,>50K
48750,58,Self-emp-inc,Doctorate,16,Never-married,Prof-specialty,Not-in-family,White,Female,0,0,99,NaN,<=50K
48773,42,Self-emp-not-inc,HS-grad,9,Divorced,Sales,Own-child,White,Male,0,0,50,NaN,<=50K
48791,39,Private,HS-grad,9,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,45,NaN,>50K


In [22]:
a['class'].value_counts()

class
<=50K    637
>50K     220
Name: count, dtype: int64

In [23]:
data['class'].value_counts()

class
<=50K    37155
>50K     11687
Name: count, dtype: int64

In [24]:
# 본국이 결측값인 사람들의 연봉구분 비율 (5만 달러 이하)
(637 / (637 + 220)) * 100

74.32905484247374

In [25]:
# 본국이 결측값인 사람들의 연봉구분 비율 (5만 달러 초과)
(220 / (637 + 220)) * 100

25.670945157526255

In [26]:
# 전체 사람들의 연봉구분 비율 (5만 달러 초과)
(11687 / (37155 + 11687)) * 100

23.928176569346054

- 전체 중 5만 달러 초과 비율과 본국이 결측값인 사람들의 5만 달러 초과 비율이 비슷하고
- 본국이 결측값인 데이터가 1.75%밖에 차지하지 않기 때문에
- 본국이 결측값인 데이터는 삭제하기로 함

In [27]:
data[data['native-country'].isna()]

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
19,40,Private,Doctorate,16,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,45,NaN,>50K
65,41,Private,Bachelors,13,Never-married,Other-service,Not-in-family,White,Female,0,0,40,NaN,<=50K
83,44,Self-emp-inc,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,99999,0,50,NaN,>50K
188,34,State-gov,Bachelors,13,Married-spouse-absent,Farming-fishing,Not-in-family,Black,Male,0,0,40,NaN,<=50K
253,42,Federal-gov,Bachelors,13,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,40,NaN,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48730,44,Self-emp-inc,Masters,14,Married-civ-spouse,Sales,Husband,White,Male,0,0,50,NaN,>50K
48750,58,Self-emp-inc,Doctorate,16,Never-married,Prof-specialty,Not-in-family,White,Female,0,0,99,NaN,<=50K
48773,42,Self-emp-not-inc,HS-grad,9,Divorced,Sales,Own-child,White,Male,0,0,50,NaN,<=50K
48791,39,Private,HS-grad,9,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,45,NaN,>50K


In [28]:
data = data.dropna()

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47985 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             47985 non-null  int64 
 1   workclass       47985 non-null  object
 2   education       47985 non-null  object
 3   education-num   47985 non-null  int64 
 4   marital-status  47985 non-null  object
 5   occupation      47985 non-null  object
 6   relationship    47985 non-null  object
 7   race            47985 non-null  object
 8   sex             47985 non-null  object
 9   capital-gain    47985 non-null  int64 
 10  capital-loss    47985 non-null  int64 
 11  hours-per-week  47985 non-null  int64 
 12  native-country  47985 non-null  object
 13  class           47985 non-null  object
dtypes: int64(5), object(9)
memory usage: 5.5+ MB


### 결측치 제거 완료

### 이상치 확인

In [30]:
data.head(3)

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K


In [31]:
data.describe()

,age,education-num,capital-gain,capital-loss,hours-per-week
count,47985.000000,47985.000000,47985.000000,47985.000000,47985.000000
mean,38.641284,10.067229,1067.032093,87.370553,40.411483
std,13.729622,2.560534,7373.256663,402.681462,12.391073
min,17.000000,1.000000,0.000000,0.000000,1.000000
25%,28.000000,9.000000,0.000000,0.000000,40.000000
50%,37.000000,10.000000,0.000000,0.000000,40.000000
75%,48.000000,12.000000,0.000000,0.000000,45.000000
max,90.000000,16.000000,99999.000000,4356.000000,99.000000


In [32]:
# capital-gain의 z score
7373 * 3 + 1067

23186

In [33]:
# capital-loss의 z score
402 * 3 + 87

1293

In [34]:
a = data[data['capital-gain'] >= 23186]

In [35]:
a.nunique()

age               59
workclass          7
education         12
education-num     12
marital-status     7
occupation        14
relationship       6
race               5
sex                2
capital-gain       6
capital-loss       1
hours-per-week    34
native-country    12
class              2
dtype: int64

In [36]:
data.groupby('occupation')['capital-gain'].describe()

,count,mean,std,min,25%,50%,75%,max
occupation,,,,,,,,
Adm-clerical,5540.0,475.911913,4122.996933,0.0,0.0,0.0,0.0,99999.0
Armed-Forces,14.0,521.285714,1950.472543,0.0,0.0,0.0,0.0,7298.0
Craft-repair,6020.0,707.891694,5340.756392,0.0,0.0,0.0,0.0,99999.0
Exec-managerial,5984.0,2216.618984,10692.954371,0.0,0.0,0.0,0.0,99999.0
Farming-fishing,1480.0,720.775000,5305.824070,0.0,0.0,0.0,0.0,99999.0
Handlers-cleaners,2046.0,284.870479,3299.556498,0.0,0.0,0.0,0.0,99999.0
Machine-op-inspct,2970.0,313.542761,2214.829103,0.0,0.0,0.0,0.0,99999.0
Other-service,4808.0,212.607737,2762.526586,0.0,0.0,0.0,0.0,99999.0
Priv-house-serv,232.0,193.146552,1718.756030,0.0,0.0,0.0,0.0,25236.0


In [37]:
gain = list(data.groupby('occupation')['capital-gain'].describe()['mean'])
gain

[475.91191335740075,
 521.2857142857143,
 707.8916943521594,
 2216.6189839572194,
 720.775,
 284.8704789833822,
 313.54276094276094,
 212.6077371048253,
 193.14655172413794,
 2740.0276298268973,
 722.2090163934427,
 1265.863350591716,
 661.1605633802817,
 448.979274611399,
 504.03619254433585]

In [38]:
result = data.groupby('occupation')['capital-gain'].describe()
oculist = result.index
oculist = list(oculist)
oculist

[' Adm-clerical',
 ' Armed-Forces',
 ' Craft-repair',
 ' Exec-managerial',
 ' Farming-fishing',
 ' Handlers-cleaners',
 ' Machine-op-inspct',
 ' Other-service',
 ' Priv-house-serv',
 ' Prof-specialty',
 ' Protective-serv',
 ' Sales',
 ' Tech-support',
 ' Transport-moving',
 'None']

#### capital-gain의 이상값 처리

In [39]:
for idx, value in zip(oculist, gain) :
    data.loc[(data['occupation'] == idx) & (data['capital-gain'] >= 23186), 'capital-gain'] = value

C:\Users\5-08\AppData\Local\Temp\ipykernel_12408\2060196591.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '475.91191335740075' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[(data['occupation'] == idx) & (data['capital-gain'] >= 23186), 'capital-gain'] = value


In [40]:
data.describe()

,age,education-num,capital-gain,capital-loss,hours-per-week
count,47985.000000,47985.000000,47985.000000,47985.000000,47985.000000
mean,38.641284,10.067229,541.867183,87.370553,40.411483
std,13.729622,2.560534,2244.473584,402.681462,12.391073
min,17.000000,1.000000,0.000000,0.000000,1.000000
25%,28.000000,9.000000,0.000000,0.000000,40.000000
50%,37.000000,10.000000,0.000000,0.000000,40.000000
75%,48.000000,12.000000,0.000000,0.000000,45.000000
max,90.000000,16.000000,22040.000000,4356.000000,99.000000


#### capital-loss의 이상값 처리
- capital-gain과 동일하게 처리

In [41]:
loss = list(data.groupby('occupation')['capital-loss'].describe()['mean'])
loss

[62.920036101083035,
 134.78571428571428,
 79.02242524916943,
 140.69485294117646,
 59.74256756756757,
 47.71163245356794,
 62.75959595959596,
 40.749584026622294,
 42.060344827586206,
 136.5033288948069,
 85.61577868852459,
 98.11039201183432,
 96.56408450704225,
 81.55224525043178,
 67.3231994209193]

In [42]:
result = data.groupby('occupation')['capital-loss'].describe()
oculist = result.index
oculist = list(oculist)
oculist

[' Adm-clerical',
 ' Armed-Forces',
 ' Craft-repair',
 ' Exec-managerial',
 ' Farming-fishing',
 ' Handlers-cleaners',
 ' Machine-op-inspct',
 ' Other-service',
 ' Priv-house-serv',
 ' Prof-specialty',
 ' Protective-serv',
 ' Sales',
 ' Tech-support',
 ' Transport-moving',
 'None']

In [43]:
for idx, value in zip(oculist, loss) :
    data.loc[(data['occupation'] == idx) & (data['capital-loss'] >= 1293), 'capital-loss'] = value

C:\Users\5-08\AppData\Local\Temp\ipykernel_12408\3032696867.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '62.920036101083035' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[(data['occupation'] == idx) & (data['capital-loss'] >= 1293), 'capital-loss'] = value


In [44]:
data.describe()

,age,education-num,capital-gain,capital-loss,hours-per-week
count,47985.000000,47985.000000,47985.000000,47985.000000,47985.000000
mean,38.641284,10.067229,541.867183,5.564742,40.411483
std,13.729622,2.560534,2244.473584,37.625742,12.391073
min,17.000000,1.000000,0.000000,0.000000,1.000000
25%,28.000000,9.000000,0.000000,0.000000,40.000000
50%,37.000000,10.000000,0.000000,0.000000,40.000000
75%,48.000000,12.000000,0.000000,0.000000,45.000000
max,90.000000,16.000000,22040.000000,1258.000000,99.000000


#### 주당 시간 이상치 확인

In [45]:
12 * 3 + 40

76

In [46]:
data[data['hours-per-week'] > 76]

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
25,45,Self-emp-not-inc,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,7298.00000,0.0,90,United-States,>50K
309,51,Self-emp-not-inc,HS-grad,9,Married-civ-spouse,Sales,Husband,Asian-Pac-Islander,Male,3103.00000,0.0,84,Vietnam,<=50K
338,36,Self-emp-inc,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.00000,0.0,80,United-States,<=50K
354,32,Private,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,0.00000,0.0,84,United-States,>50K
357,50,Self-emp-not-inc,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,2740.02763,0.0,80,United-States,>50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48438,45,Self-emp-not-inc,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.00000,0.0,80,United-States,<=50K
48672,55,Private,HS-grad,9,Separated,Protective-serv,Not-in-family,White,Male,0.00000,0.0,81,United-States,<=50K
48683,42,Private,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0.00000,0.0,89,United-States,>50K
48757,35,Private,12th,8,Married-civ-spouse,Craft-repair,Husband,White,Male,0.00000,0.0,84,United-States,<=50K


In [47]:
data[data['hours-per-week'] > 76]['class'].value_counts(sort = False)

class
>50K     182
<=50K    358
Name: count, dtype: int64

In [48]:
data[data['hours-per-week'] <= 76]['class'].value_counts(sort = False)

class
<=50K    36160
>50K     11285
Name: count, dtype: int64

In [49]:
182 / (182 + 358) * 100

33.7037037037037

In [50]:
11285 / (11285 + 36160) * 100

23.785435767731055

- 이상치 기준 일을 많이 하면 연봉이 5만 달러 초과인 사람은 33퍼
- 이상치 기준 일을 적게 하는 사람보다 10%가 더 많음

In [51]:
data[data['hours-per-week'] > 76]['workclass'].value_counts()

workclass
 Private             248
 Self-emp-not-inc    144
 Self-emp-inc         66
None                  32
 Local-gov            25
 State-gov            16
 Federal-gov           9
Name: count, dtype: int64

- 일을 많이 하는 사람들은 어떤 유형일까 봤는데
- 대부분 자기 회사를 운영하는 사람들임
- 고로 이상값이라고 생각한 값들은 이상값이 아닐 확률이 큼

- Self-emp-not-inc : 비상장기업 & 자기 회사
- Self-emp-not-inc : 상장기업 & 자기 회사

In [52]:
data[data['hours-per-week'] > 76].groupby('workclass')['class'].value_counts(sort = False)

workclass          class
 Federal-gov       <=50K      5
                   >50K       4
 Local-gov         <=50K     14
                   >50K      11
 Private           <=50K    153
                   >50K      95
 Self-emp-inc      <=50K     32
                   >50K      34
 Self-emp-not-inc  <=50K    113
                   >50K      31
 State-gov         <=50K     14
                   >50K       2
None               <=50K     27
                   >50K       5
Name: count, dtype: int64

#### 주당 노동시간은 이상값 처리 x

#### 범주형 자료 숫자형으로 전환

- chatgpt에 학력을 어떻게 숫자형 데이터로 바꾸면 좋을지 물었으니까 보고 하기
- 가족 관계는 get-dummies 활용하기
- relationship, sex의 경우도 get-dummies 활용하기
- 국적은 국적 별 class를 계산하고 그 순서대로 숫자로 처리하기

#### 학력 데이터를 숫자형으로 전환하기

In [53]:
# 학력 순서에 따른 숫자 변환 딕셔너리 생성
education_mapping = {
    'Preschool': 1,
    '1st-4th': 2,
    '5th-6th': 3,
    '7th-8th': 4,
    '9th': 5,
    '10th': 6,
    '11th': 7,
    '12th': 8,
    'HS-grad': 9,
    'Some-college': 10,
    'Assoc-acdm': 11,
    'Assoc-voc': 12,
    'Bachelors': 13,
    'Masters': 14,
    'Prof-school': 15,
    'Doctorate': 16
}

In [54]:
data.loc[:, 'education'] = data['education'].str.strip().map(education_mapping)

In [55]:
data.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,7,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40,United-States,<=50K
1,38,Private,9,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50,United-States,<=50K
2,28,Local-gov,11,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40,United-States,>50K
3,44,Private,10,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40,United-States,>50K
4,18,None,10,10,Never-married,None,Own-child,White,Female,0.0,0.0,30,United-States,<=50K


#### 학력 숫자형으로 처리 완료

#### 가족형태가 class와 상관이 있는지 보기

In [56]:
a = data.groupby('marital-status')['class'].describe()
a

,count,unique,top,freq
marital-status,,,,
Divorced,6561,2,<=50K,5897
Married-AF-spouse,37,2,<=50K,23
Married-civ-spouse,21971,2,<=50K,12174
Married-spouse-absent,595,2,<=50K,540
Never-married,15816,2,<=50K,15105
Separated,1507,2,<=50K,1408
Widowed,1498,2,<=50K,1371


In [57]:
a['freq'] / a['count'] * 100

marital-status
Divorced                 89.879592
Married-AF-spouse        62.162162
Married-civ-spouse       55.409403
Married-spouse-absent    90.756303
Never-married            95.504552
Separated                93.430657
Widowed                  91.522029
dtype: object

- 가족형태와 class 간의 관계가 있는지 봤을 때
- 결혼하고 가정을 이루고 있는 경우 3~40%가 5만 달러 넘고
- 남편과 아내가 둘 중 하나만 있는 경우나 독신 가정인 경우엔 5만 달러 넘는 비율이 10% 미만
- 고로 가정형태가 연봉에 영향을 준다고 판단할 수 있음

#### 자녀유무에 따라 연봉 보기

In [58]:
b = data.groupby('relationship')['class'].describe()
b

,count,unique,top,freq
relationship,,,,
Husband,19371,2,<=50K,10689
Not-in-family,12353,2,<=50K,11102
Other-relative,1452,2,<=50K,1401
Own-child,7491,2,<=50K,7384
Unmarried,5041,2,<=50K,4736
Wife,2277,2,<=50K,1206


In [59]:
b['freq'] / b['count'] * 100

relationship
Husband           55.180424
Not-in-family     89.872905
Other-relative    96.487603
Own-child         98.571619
Unmarried         93.949613
Wife              52.964427
dtype: object

- 자녀가 없이 가정이 이루어진 경우(husband, wife) 5만 달러 넘는 비율이 45% 정도 됨
- 반면, 자녀가 있거나(own-child), 부부가 이뤄지지 않은(Not-in-family, Other-relative, Unmarried)의 경우 5만 달러가 넘는 비율이 10%
- 유의미한 결과의 차이가 있다고 판단해 class에 영향을 준다고 판단

In [60]:
race = data.groupby('race')['class'].describe()
race

,count,unique,top,freq
race,,,,
Amer-Indian-Eskimo,470,2,<=50K,415
Asian-Pac-Islander,1391,2,<=50K,1016
Black,4553,2,<=50K,4006
Other,382,2,<=50K,335
White,41189,2,<=50K,30746


In [61]:
race['freq'] / race['count'] * 100

race
Amer-Indian-Eskimo    88.297872
Asian-Pac-Islander    73.040978
Black                 87.985943
Other                 87.696335
White                 74.646143
dtype: object

In [62]:
c = data.groupby(['race','workclass'])['class'].describe()
c

count unique     top   freq
race               workclass                                     
Amer-Indian-Eskimo  Federal-gov          33      2   <=50K     29
                    Local-gov            65      2   <=50K     59
                    Private             278      2   <=50K    250
                    Self-emp-inc          2      2   <=50K      1
                    Self-emp-not-inc     34      2   <=50K     28
                    State-gov            23      2   <=50K     15
                   None                  35      2   <=50K     33
Asian-Pac-Islander  Federal-gov          58      2    >50K     34
                    Local-gov            58      2   <=50K     38
                    Private             955      2   <=50K    717
                    Self-emp-inc         56      2    >50K     30
                    Self-emp-not-inc     90      2   <=50K     64
                    State-gov            85      2   <=50K     64
                    Without-pay           1      1   <=50K      1
                   None                  88      2   <=50K     82
Black               Federal-gov         254      2   <=50K    205
                    Local-gov           427      2   <=50K    336
                    Never-worked          3      1   <=50K      3
                    Private            3147      2   <=50K   2828
                    Self-emp-inc         38      2   <=50K     23
                    Self-emp-not-inc    128      2   <=50K    101
                    State-gov           233      2   <=50K    200
                    Without-pay           1      1   <=50K      1
                   None                 322      2   <=50K    309
Other               Federal-gov          10      2   <=50K      8
                    Local-gov            12      2   <=50K      7
                    Private             298      2   <=50K    272
                    Self-emp-inc          5      2   <=50K      3
                    Self-emp-not-inc     14      2   <=50K      8
                    State-gov            14      2   <=50K     10
                   None                  29      2   <=50K     27
White               Federal-gov        1051      2   <=50K    591
                    Local-gov          2538      2   <=50K   1745
                    Never-worked          7      1   <=50K      7
                    Private           28629      2   <=50K  21989
                    Self-emp-inc       1545      2    >50K    864
                    Self-emp-not-inc   3530      2   <=50K   2536
                    State-gov          1591      2   <=50K   1137
                    Without-pay          19      2   <=50K     17
                   None                2279      2   <=50K   2043

In [63]:
# 5만 달러 초과
# api_federal, api_sei, white_sei는 5만달러 미만
100 - (c['freq'] / c['count'] * 100)

race                workclass        
Amer-Indian-Eskimo   Federal-gov         12.121212
                     Local-gov            9.230769
                     Private             10.071942
                     Self-emp-inc             50.0
                     Self-emp-not-inc    17.647059
                     State-gov           34.782609
                    None                  5.714286
Asian-Pac-Islander   Federal-gov          41.37931
                     Local-gov           34.482759
                     Private             24.921466
                     Self-emp-inc        46.428571
                     Self-emp-not-inc    28.888889
                     State-gov           24.705882
                     Without-pay               0.0
                    None                  6.818182
Black                Federal-gov         19.291339
                     Local-gov           21.311475
                     Never-worked              0.0
                     Private             10.

#### 인종은 연봉에 거의 영향을 미치지 않는다고 판단했는데 인종, 직업별 연봉을 봤을 때 차이가 눈에 띄게 있다고 판단
- 인종 컬럼 남기기

In [64]:
coun = data.groupby('native-country')['class'].describe()
coun

,count,unique,top,freq
native-country,,,,
Cambodia,28,2,<=50K,19
Canada,182,2,<=50K,119
China,122,2,<=50K,86
Columbia,85,2,<=50K,81
Cuba,138,2,<=50K,104
Dominican-Republic,103,2,<=50K,98
Ecuador,45,2,<=50K,39
El-Salvador,155,2,<=50K,144
England,127,2,<=50K,80


In [65]:
# 5만 달러 초과인 국적
100 - (coun['freq'] / coun['count'] * 100)

native-country
Cambodia                      32.142857
Canada                        34.615385
China                         29.508197
Columbia                       4.705882
Cuba                          24.637681
Dominican-Republic             4.854369
Ecuador                       13.333333
El-Salvador                    7.096774
England                       37.007874
France                        42.105263
Germany                        28.15534
Greece                        36.734694
Guatemala                      3.409091
Haiti                              12.0
Holand-Netherlands                  0.0
Honduras                           10.0
Hong                          26.666667
Hungary                       31.578947
India                         41.059603
Iran                          37.288136
Ireland                        29.72973
Italy                         32.380952
Jamaica                       14.150943
Japan                         34.782609
Laos                     

- 별 다른 걸 발견 못해서 대륙 별로 구분지어 보기로 함

In [66]:
continent_mapping = {
    ' Cambodia': 'Asia', ' Canada': 'North America', ' China': 'Asia', ' Columbia': 'South America',
    ' Cuba' : 'North America', ' Dominican-Republic' : 'North America', ' Ecuador': 'South America',
    ' El-Salvador' : 'Central America', ' England': 'Europe', ' France': 'Europe', ' Germany': 'Europe',
    ' Greece': 'Europe', ' Guatemala' : 'Central America', ' Haiti' : 'Central America', ' Holand-Netherlands': 'Europe',
    ' Honduras' : 'Central America', ' Hong': 'Asia', ' Hungary': 'Europe', ' India': 'Asia', ' Iran': 'Asia', ' Ireland': 'Europe',
    ' Italy': 'Europe', ' Jamaica' : 'North America', ' Japan': 'Asia', ' Laos': 'Asia', ' Mexico': 'North America',
    ' Nicaragua' : 'Central America', ' Outlying-US' : 'North America', ' Peru': 'South America', ' Philippines': 'Asia',
    ' Poland': 'Europe', ' Portugal': 'Europe', ' Puerto-Rico' : 'North America', ' South': 'Asia', ' Taiwan': 'Asia',
    ' Thailand': 'Asia', ' Trinadad&Tobago': 'North America', ' United-States': 'North America', ' Vietnam': 'Asia',
    ' Yugoslavia': 'Europe'
}

In [67]:
data.loc[:,'continent'] = data['native-country'].map(continent_mapping)

C:\Users\5-08\AppData\Local\Temp\ipykernel_12408\3716643931.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:,'continent'] = data['native-country'].map(continent_mapping)


In [68]:
data.head(2)

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class,continent
0,25,Private,7,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40,United-States,<=50K,North America
1,38,Private,9,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50,United-States,<=50K,North America


In [69]:
data.groupby('continent')['class'].value_counts(sort = False)

continent        class
Asia             <=50K      782
                 >50K       314
Central America  <=50K      359
                 >50K        28
Europe           <=50K      532
                 >50K       227
North America    <=50K    34643
                 >50K     10880
South America    <=50K      162
                 >50K        14
Name: count, dtype: int64

#### marital-status랑 occupation, relationship, race, sex, class, continent를 get_dummies로 돌리기

In [70]:
data_dm = pd.get_dummies(data, columns = ['workclass', "marital-status", "occupation", 'relationship', 'race', 'sex', 'continent'], drop_first = True)

In [71]:
data_dm.head(2)

,age,education,education-num,capital-gain,capital-loss,hours-per-week,native-country,class,workclass_ Local-gov,workclass_ Never-worked,...,relationship_ Wife,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,sex_ Male,continent_Central America,continent_Europe,continent_North America,continent_South America
0,25,7,7,0.0,0.0,40,United-States,<=50K,False,False,...,False,False,True,False,False,True,False,False,True,False
1,38,9,9,0.0,0.0,50,United-States,<=50K,False,False,...,False,False,False,False,True,True,False,False,True,False


In [72]:
from sklearn.preprocessing import LabelEncoder

In [73]:
le = LabelEncoder()
data_dm['class'] = le.fit_transform(data_dm['class'])
data_dm['class']

0        0
1        0
2        1
3        1
4        0
        ..
48837    0
48838    1
48839    0
48840    0
48841    1
Name: class, Length: 47985, dtype: int32

In [74]:
data_dm.head(2)

,age,education,education-num,capital-gain,capital-loss,hours-per-week,native-country,class,workclass_ Local-gov,workclass_ Never-worked,...,relationship_ Wife,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,sex_ Male,continent_Central America,continent_Europe,continent_North America,continent_South America
0,25,7,7,0.0,0.0,40,United-States,0,False,False,...,False,False,True,False,False,True,False,False,True,False
1,38,9,9,0.0,0.0,50,United-States,0,False,False,...,False,False,False,False,True,True,False,False,True,False


In [75]:
data_dm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47985 entries, 0 to 48841
Data columns (total 50 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   age                                    47985 non-null  int64  
 1   education                              47985 non-null  object 
 2   education-num                          47985 non-null  int64  
 3   capital-gain                           47985 non-null  float64
 4   capital-loss                           47985 non-null  float64
 5   hours-per-week                         47985 non-null  int64  
 6   native-country                         47985 non-null  object 
 7   class                                  47985 non-null  int32  
 8   workclass_ Local-gov                   47985 non-null  bool   
 9   workclass_ Never-worked                47985 non-null  bool   
 10  workclass_ Private                     47985 non-null  bool   
 11  workcla

In [76]:
data_dm['education'] = data_dm['education'].astype(int)

In [77]:
data_dm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47985 entries, 0 to 48841
Data columns (total 50 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   age                                    47985 non-null  int64  
 1   education                              47985 non-null  int32  
 2   education-num                          47985 non-null  int64  
 3   capital-gain                           47985 non-null  float64
 4   capital-loss                           47985 non-null  float64
 5   hours-per-week                         47985 non-null  int64  
 6   native-country                         47985 non-null  object 
 7   class                                  47985 non-null  int32  
 8   workclass_ Local-gov                   47985 non-null  bool   
 9   workclass_ Never-worked                47985 non-null  bool   
 10  workclass_ Private                     47985 non-null  bool   
 11  workcla

#### 본국을 대륙으로 전환
- data2에는 본국을 제외하고 해보기

In [78]:
data_dm2 = data_dm.copy()

In [79]:
data_dm2 = data_dm2.drop('native-country', axis = 1)

In [80]:
data_dm2.head(2)

,age,education,education-num,capital-gain,capital-loss,hours-per-week,class,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,...,relationship_ Wife,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,sex_ Male,continent_Central America,continent_Europe,continent_North America,continent_South America
0,25,7,7,0.0,0.0,40,0,False,False,True,...,False,False,True,False,False,True,False,False,True,False
1,38,9,9,0.0,0.0,50,0,False,False,True,...,False,False,False,False,True,True,False,False,True,False


### 의사결정나무 머신러닝 돌리기

In [81]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [82]:
X = data_dm2.drop('class', axis = 1)
y = data_dm2['class']

In [83]:
y.value_counts()

class
0    36518
1    11467
Name: count, dtype: int64

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 8)

In [85]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
pred = dtc.predict(X_test)
print(accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

0.8183807439824945
              precision    recall  f1-score   support

           0       0.88      0.88      0.88      7337
           1       0.62      0.60      0.61      2260

    accuracy                           0.82      9597
   macro avg       0.75      0.74      0.75      9597
weighted avg       0.82      0.82      0.82      9597



In [86]:
for i in range(1, 10) : 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = i/10, random_state = 8)
    dtc = DecisionTreeClassifier()
    dtc.fit(X_train, y_train)
    pred = dtc.predict(X_test)
    print(i, accuracy_score(y_test, pred))
    print(classification_report(y_test, pred))

1 0.8130860595957491
              precision    recall  f1-score   support

           0       0.88      0.88      0.88      3706
           1       0.59      0.60      0.59      1093

    accuracy                           0.81      4799
   macro avg       0.73      0.74      0.74      4799
weighted avg       0.81      0.81      0.81      4799

2 0.8165051578618319
              precision    recall  f1-score   support

           0       0.88      0.88      0.88      7337
           1       0.61      0.61      0.61      2260

    accuracy                           0.82      9597
   macro avg       0.75      0.74      0.74      9597
weighted avg       0.82      0.82      0.82      9597

3 0.8129341483745485
              precision    recall  f1-score   support

           0       0.88      0.88      0.88     10989
           1       0.61      0.59      0.60      3407

    accuracy                           0.81     14396
   macro avg       0.74      0.74      0.74     14396
weighted av

In [87]:
for i in range(1,10) :
    dtc = DecisionTreeClassifier(max_depth = i, random_state = 8)
    dtc.fit(X_train, y_train)
    pred = dtc.predict(X_test)
    print(i, accuracy_score(y_test, pred))
    print(i, classification_report(y_test,pred))

1 0.7607844953342441
1               precision    recall  f1-score   support

           0       0.76      1.00      0.86     32856
           1       0.00      0.00      0.00     10331

    accuracy                           0.76     43187
   macro avg       0.38      0.50      0.43     43187
weighted avg       0.58      0.76      0.66     43187

2 0.8141801931136685
2               precision    recall  f1-score   support

           0       0.89      0.86      0.88     32856
           1       0.60      0.66      0.63     10331

    accuracy                           0.81     43187
   macro avg       0.75      0.76      0.75     43187
weighted avg       0.82      0.81      0.82     43187

3 0.8200847477250098
3               precision    recall  f1-score   support

           0       0.90      0.86      0.88     32856
           1       0.61      0.71      0.65     10331

    accuracy                           0.82     43187
   macro avg       0.75      0.78      0.77     43187
weigh

C:\Users\5-08\miniforge3\envs\fintech\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\5-08\miniforge3\envs\fintech\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\5-08\miniforge3\envs\fintech\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


5 0.8410632829323639
5               precision    recall  f1-score   support

           0       0.85      0.96      0.90     32856
           1       0.77      0.48      0.59     10331

    accuracy                           0.84     43187
   macro avg       0.81      0.72      0.75     43187
weighted avg       0.83      0.84      0.83     43187

6 0.8440271377960961
6               precision    recall  f1-score   support

           0       0.86      0.95      0.90     32856
           1       0.76      0.51      0.61     10331

    accuracy                           0.84     43187
   macro avg       0.81      0.73      0.76     43187
weighted avg       0.84      0.84      0.83     43187

7 0.8414569199064533
7               precision    recall  f1-score   support

           0       0.86      0.94      0.90     32856
           1       0.74      0.52      0.61     10331

    accuracy                           0.84     43187
   macro avg       0.80      0.73      0.76     43187
weigh

In [88]:
for i in range(1,10) :
    dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = i, random_state = 8)
    dtc.fit(X_train, y_train)
    pred = dtc.predict(X_test)
    print(i, accuracy_score(y_test, pred))
    print(i, classification_report(y_test,pred))

1 0.7607844953342441
1               precision    recall  f1-score   support

           0       0.76      1.00      0.86     32856
           1       0.00      0.00      0.00     10331

    accuracy                           0.76     43187
   macro avg       0.38      0.50      0.43     43187
weighted avg       0.58      0.76      0.66     43187

2 0.7986199550790747
2               precision    recall  f1-score   support

           0       0.79      1.00      0.88     32856
           1       0.99      0.16      0.28     10331

    accuracy                           0.80     43187
   macro avg       0.89      0.58      0.58     43187
weighted avg       0.84      0.80      0.74     43187

3 0.8413874545580846
3               precision    recall  f1-score   support

           0       0.86      0.95      0.90     32856
           1       0.76      0.50      0.60     10331

    accuracy                           0.84     43187
   macro avg       0.81      0.72      0.75     43187
weigh

C:\Users\5-08\miniforge3\envs\fintech\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\5-08\miniforge3\envs\fintech\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\5-08\miniforge3\envs\fintech\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


5 0.8404844050292912
5               precision    recall  f1-score   support

           0       0.85      0.95      0.90     32856
           1       0.77      0.48      0.59     10331

    accuracy                           0.84     43187
   macro avg       0.81      0.72      0.74     43187
weighted avg       0.83      0.84      0.83     43187

6 0.8440271377960961
6               precision    recall  f1-score   support

           0       0.86      0.95      0.90     32856
           1       0.77      0.50      0.61     10331

    accuracy                           0.84     43187
   macro avg       0.81      0.73      0.75     43187
weighted avg       0.84      0.84      0.83     43187

7 0.8439345173316044
7               precision    recall  f1-score   support

           0       0.86      0.95      0.90     32856
           1       0.76      0.51      0.61     10331

    accuracy                           0.84     43187
   macro avg       0.81      0.73      0.76     43187
weigh

In [89]:
for i in range(1,10) :
    dtc = DecisionTreeClassifier(criterion = 'log_loss', max_depth = i, random_state = 8)
    dtc.fit(X_train, y_train)
    pred = dtc.predict(X_test)
    print(i, accuracy_score(y_test, pred))
    print(i, classification_report(y_test,pred))

1 0.7607844953342441
1               precision    recall  f1-score   support

           0       0.76      1.00      0.86     32856
           1       0.00      0.00      0.00     10331

    accuracy                           0.76     43187
   macro avg       0.38      0.50      0.43     43187
weighted avg       0.58      0.76      0.66     43187

2 0.7986199550790747
2               precision    recall  f1-score   support

           0       0.79      1.00      0.88     32856
           1       0.99      0.16      0.28     10331

    accuracy                           0.80     43187
   macro avg       0.89      0.58      0.58     43187
weighted avg       0.84      0.80      0.74     43187

3 0.8413874545580846
3               precision    recall  f1-score   support

           0       0.86      0.95      0.90     32856
           1       0.76      0.50      0.60     10331

    accuracy                           0.84     43187
   macro avg       0.81      0.72      0.75     43187
weigh

C:\Users\5-08\miniforge3\envs\fintech\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\5-08\miniforge3\envs\fintech\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\5-08\miniforge3\envs\fintech\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


5 0.8404844050292912
5               precision    recall  f1-score   support

           0       0.85      0.95      0.90     32856
           1       0.77      0.48      0.59     10331

    accuracy                           0.84     43187
   macro avg       0.81      0.72      0.74     43187
weighted avg       0.83      0.84      0.83     43187

6 0.8440271377960961
6               precision    recall  f1-score   support

           0       0.86      0.95      0.90     32856
           1       0.77      0.50      0.61     10331

    accuracy                           0.84     43187
   macro avg       0.81      0.73      0.75     43187
weighted avg       0.84      0.84      0.83     43187

7 0.8439345173316044
7               precision    recall  f1-score   support

           0       0.86      0.95      0.90     32856
           1       0.76      0.51      0.61     10331

    accuracy                           0.84     43187
   macro avg       0.81      0.73      0.76     43187
weigh

### 배깅으로 해보기

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 8)

In [91]:
from sklearn.ensemble import BaggingClassifier

In [92]:
ba = BaggingClassifier(n_estimators = 1000, n_jobs = 10, random_state = 8)
ba.fit(X_train, y_train)
pred = ba.predict(X_test)
print(accuracy_score(y_test, pred))
print(classification_report(y_test,pred))

0.8414087735750756
              precision    recall  f1-score   support

           0       0.89      0.91      0.90      7337
           1       0.68      0.62      0.65      2260

    accuracy                           0.84      9597
   macro avg       0.78      0.76      0.77      9597
weighted avg       0.84      0.84      0.84      9597



## 부스팅으로 해보기

### 에이다 부스팅

In [93]:
from sklearn.ensemble import AdaBoostClassifier

In [94]:
ada = AdaBoostClassifier(n_estimators = 1000, learning_rate = 0.1, random_state = 8)
ada.fit(X_train, y_train)
pred = ada.predict(X_test)
print(accuracy_score(y_test, pred))
print(classification_report(y_test,pred))

C:\Users\5-08\miniforge3\envs\fintech\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


KeyboardInterrupt: 

### 그래디언트부스팅

In [95]:
from sklearn.ensemble import GradientBoostingClassifier

In [96]:
gbc = GradientBoostingClassifier(n_estimators = 1000, random_state = 8, learning_rate = 0.1)
gbc.fit(X_train, y_train)
pred = gbc.predict(X_test)
print(accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

0.8726685422527873
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      7337
           1       0.76      0.66      0.71      2260

    accuracy                           0.87      9597
   macro avg       0.83      0.80      0.81      9597
weighted avg       0.87      0.87      0.87      9597



In [103]:
for i in range(1, 10) : 
    gbc = GradientBoostingClassifier(n_estimators = 1000, random_state = 8, learning_rate = 0.1, max_depth = i)
    gbc.fit(X_train, y_train)
    pred = gbc.predict(X_test)
    print(i, accuracy_score(y_test, pred))
    print(i, classification_report(y_test, pred))

1 0.8634990101073252
1               precision    recall  f1-score   support

           0       0.89      0.94      0.91      7337
           1       0.76      0.62      0.68      2260

    accuracy                           0.86      9597
   macro avg       0.82      0.78      0.80      9597
weighted avg       0.86      0.86      0.86      9597

2 0.8711055538189018
2               precision    recall  f1-score   support

           0       0.90      0.94      0.92      7337
           1       0.76      0.66      0.71      2260

    accuracy                           0.87      9597
   macro avg       0.83      0.80      0.81      9597
weighted avg       0.87      0.87      0.87      9597

3 0.8726685422527873
3               precision    recall  f1-score   support

           0       0.90      0.94      0.92      7337
           1       0.76      0.66      0.71      2260

    accuracy                           0.87      9597
   macro avg       0.83      0.80      0.81      9597
weigh

## 정확도가 가장 좋은 것
- gradientboost 사용
- max_depth = 3
- test-size = 0.2

gbc = GradientBoostingClassifier(n_estimators = 1000, random_state = 8, learning_rate = 0.1, max_depth = 3) <br>
gbc.fit(X_train, y_train) <br>
pred = gbc.predict(X_test) <br>
print(i, accuracy_score(y_test, pred)) <br>
print(i, classification_report(y_test, pred)) <br>

### xgboost

In [ ]:
X = data_dm2.drop('class', axis = 1)
y = data_dm2['class']

In [145]:
X_train.head()

,age,education,education-num,capital-gain,capital-loss,hours-per-week,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,...,relationship_ Wife,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,sex_ Male,continent_Central America,continent_Europe,continent_North America,continent_South America
1288,28,12,11,0.0,0.0,50,False,False,True,False,...,False,False,False,False,True,True,False,False,True,False
535,30,14,14,0.0,0.0,30,False,False,True,False,...,True,False,False,False,True,False,False,False,True,False
36240,45,5,5,0.0,0.0,40,False,False,True,False,...,False,True,False,False,False,True,False,False,False,False
409,38,13,13,0.0,0.0,24,False,False,True,False,...,True,False,False,False,True,False,False,False,True,False
21935,25,13,13,0.0,0.0,40,False,False,True,False,...,False,False,False,False,True,True,False,False,True,False


In [155]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 8)

In [97]:
from xgboost import XGBClassifier

In [104]:
xgb = XGBClassifier(n_estimators = 1000, learning_rate = 0.1, max_depth = 3, random_state = 8, n_jobs = 10)
xgb.fit(X_train, y_train)
pred = xgb.predict(X_test)
print(accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

0.8704803584453475
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      7337
           1       0.76      0.66      0.71      2260

    accuracy                           0.87      9597
   macro avg       0.83      0.80      0.81      9597
weighted avg       0.87      0.87      0.87      9597



### lightgbm

In [99]:
from lightgbm import LGBMClassifier

In [105]:
lgbm = LGBMClassifier(n_estimators = 1000, max_depth = 3, learning_rate = 0.1, random_state = 8, n_jobs = 10)
lgbm.fit(X_train, y_train)
pred = lgbm.predict(X_test)
print(accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 9207, number of negative: 29181
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001233 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 409
[LightGBM] [Info] Number of data points in the train set: 38388, number of used features: 45
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.239841 -> initscore=-1.153554
[LightGBM] [Info] Start training from score -1.153554
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

0.8715223507346046
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      7337
           1       0.76      0.66      0.71      2260

    accuracy                           0.87      9597
   macro avg       0.83      0.80      0.81      9597
weighted avg       0.87      0.87      0.87      9597



### randomforest

In [101]:
from sklearn.ensemble import RandomForestClassifier

In [102]:
rfc = RandomForestClassifier(n_estimators = 1000, random_state = 8, n_jobs = 10)
rfc.fit(X_train, y_train)
pred = rfc.predict(X_test)
print(accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

0.8434927581535897
              precision    recall  f1-score   support

           0       0.89      0.91      0.90      7337
           1       0.69      0.62      0.65      2260

    accuracy                           0.84      9597
   macro avg       0.79      0.77      0.78      9597
weighted avg       0.84      0.84      0.84      9597



## education 컬럼 삭제해보기
- education이랑 education_num이랑 비슷함

In [106]:
data_dm3 = data_dm2.copy()

In [107]:
data_dm3.head()

,age,education,education-num,capital-gain,capital-loss,hours-per-week,class,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,...,relationship_ Wife,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,sex_ Male,continent_Central America,continent_Europe,continent_North America,continent_South America
0,25,7,7,0.0,0.0,40,0,False,False,True,...,False,False,True,False,False,True,False,False,True,False
1,38,9,9,0.0,0.0,50,0,False,False,True,...,False,False,False,False,True,True,False,False,True,False
2,28,11,12,0.0,0.0,40,1,True,False,False,...,False,False,False,False,True,True,False,False,True,False
3,44,10,10,7688.0,0.0,40,1,False,False,True,...,False,False,True,False,False,True,False,False,True,False
4,18,10,10,0.0,0.0,30,0,False,False,False,...,False,False,False,False,True,False,False,False,True,False


In [108]:
data_dm3 = data_dm3.drop('education', axis = 1)

In [109]:
X = data_dm3.drop('class', axis = 1)
y = data_dm3['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 8)

In [110]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
pred = dtc.predict(X_test)
print(accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

0.817859747837866
              precision    recall  f1-score   support

           0       0.88      0.88      0.88      7337
           1       0.61      0.61      0.61      2260

    accuracy                           0.82      9597
   macro avg       0.75      0.75      0.75      9597
weighted avg       0.82      0.82      0.82      9597



In [111]:
# 그래디언트 부스팅
gbc = GradientBoostingClassifier(n_estimators = 1000, random_state = 8, learning_rate = 0.1)
gbc.fit(X_train, y_train)
pred = gbc.predict(X_test)
print(accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

0.8712097530478274
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      7337
           1       0.76      0.66      0.71      2260

    accuracy                           0.87      9597
   macro avg       0.83      0.80      0.81      9597
weighted avg       0.87      0.87      0.87      9597



In [113]:
# 랜덤포레스트
rfc = RandomForestClassifier(n_estimators = 1000, random_state = 8, n_jobs = 10, max_depth = 3)
rfc.fit(X_train, y_train)
pred = rfc.predict(X_test)
print(accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

0.8023340627279358
              precision    recall  f1-score   support

           0       0.80      1.00      0.89      7337
           1       0.98      0.16      0.28      2260

    accuracy                           0.80      9597
   macro avg       0.89      0.58      0.58      9597
weighted avg       0.84      0.80      0.74      9597

